In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
import surp
import arya

In [ ]:
from scipy.integrate import quad

# loading model

In [ ]:
model_name = "conroy22"

In [ ]:
fiducial = surp.ViceModel.from_file(f"../../models/fiducial/{model_name}/model.json")

In [ ]:
params = surp.simulation.parameters.MWParams.from_file(f"../../models/fiducial/{model_name}/params.toml")

In [ ]:
model  = surp.simulation.create_model(params)

In [ ]:
R_MAX = 15.5
h = fiducial.history[fiducial.history.R < R_MAX]

In [ ]:
t_end = np.max(h.time)

In [ ]:
filt = h.time == t_end
h_end = h[filt]

# Star Formation History
Should go as the two part power law

In [ ]:
plt.scatter(h.time, h.sfr, c=h.R, s=0.01)
plt.xlabel("time")
plt.ylabel("sfr")
plt.colorbar(label="radius")

In [ ]:
Rs = 0.05 + np.arange(4, 13, 2)

hm = arya.HueMap((3, 15))
for R in Rs:
    tau_rise = 2
    filt = np.isclose(h.R, R)
    h1 = h[filt]
    plt.plot(h1.time, np.log10(h1.sfr ), color=hm(R), label=R)
    
plt.xlabel("time / Gyr")
plt.ylabel("log sfr rate")
arya.Legend(-1)
plt.ylim(-4, -0.5)

In [ ]:
for R, tau_sfh in [(3.05, 7.5), (7.05, 14), (14.05, 35)]:
    tau_rise = 2
    filt = np.isclose(h.R, R)
    h1 = h[filt]
    plt.plot(h1.time, h1.ifr / np.max(h1.ifr), label=R)
plt.xlabel("time / Gyr")
plt.ylabel("peak-normalized infall rate")
plt.legend(title="radius")

In [ ]:
plt.scatter(h_end.R, h_end.sfr)
plt.ylabel("star formation rate ")
plt.xlabel("radius / kpc")

In [ ]:
dR = np.diff(np.sort(h_end.R))[1]

In [ ]:
A = (2*np.pi * h_end.R * dR).values
plt.scatter(h_end.R, h_end.mstar / A)
#plt.plot(h_end.R, h_end.mgas / A)

plt.yscale("log")
#plt.xscale("log")
plt.scatter(h_end.R, 1e9*(np.exp(-h_end.R / 2.5) + 0.27*np.exp(-h_end.R / 2.0)))
plt.ylabel(r"$\Sigma_\star$")
plt.xlabel("R")


star formation densities are okay. divergence at centre is likely a result of migration but may not even be that unphysical.

In [ ]:
print("%0.2e" % np.sum(h_end.mstar)) # total mass should be 5.17 x 10^10 solar masses

# SFR Law

In [ ]:
Σgas = h_end.mgas/A

1/(h_end.sfr / A / Σgas * 1e9)[np.isclose(h_end.R, 7.05)]

In [ ]:
Σgas = h_end.mgas/A
yr_per_Gyr = 1e9
tau_star = h_end.mgas /(h_end.sfr * yr_per_Gyr)
plt.scatter(h_end.R, tau_star)

plt.xlabel(r"R / kpc")
plt.ylabel(r"$\tau_\star\,$/Gyr")
plt.title("z = 0")


In [ ]:
h_solar = h[h.R == 8.05]

In [ ]:
tau_star = h_solar.mgas /(h_solar.sfr * yr_per_Gyr)

plt.plot(h_solar.time, tau_star)
plt.ylim(0, 60)
plt.xlabel("time")
plt.ylabel("tau star / Gyr")

# Outflows and yields

In [ ]:
plt.scatter(h_end.R, h_end.eta_0)
plt.xlabel("R")
plt.ylabel(r"$\eta$")
plt.yscale("log")

In [ ]:
from surp.simulation.properties import MH_grad

In [ ]:
plt.scatter(h_end.R, h_end.M_H)
y = MH_grad(params)(h_end.R)
plt.scatter(h_end.R, y)

plt.xlim(0, 15)
plt.ylim(-0.7, 0.7)
plt.xlabel("R")
plt.ylabel(r"[M/H]")

In [ ]:
filt = h.R < 5
df = h[filt]
plt.scatter(df.time, df.MG_H, s=0.3, alpha=0.1, c=df.R)
plt.xlim(0, 15)
plt.ylim(-0.5, 0.7)
plt.xlabel("time")
plt.ylabel("[Mg/H]")
plt.colorbar()

In [ ]:
filt = h.R < 10
df = h[filt]
plt.scatter(df.FE_H, df.MG_FE, s=0.3, alpha=0.1, c=df.R)
plt.xlim(-4, 2)
plt.ylim(-0.5, 0.7)
plt.xlabel("[Fe/H]")
plt.ylabel("[Mg/Fe]")
plt.colorbar(label="radius")